# End-to-End Meta-Analysis with PyMeta

This notebook demonstrates a complete meta-analysis workflow using PyMeta, from data loading to publication-ready results.

## 1. Setup and Data Loading

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import PyMeta
import pymeta as pm

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

In [ ]:
# Load example data
data = pm.load_example_data('binary_outcomes')
print(f"Loaded {len(data)} studies")
data.head()

## 2. Data Exploration

In [ ]:
# Basic data summary
print("Study characteristics:")
print(f"- Publication years: {data['year'].min()} - {data['year'].max()}")
print(f"- Sample sizes: {data['total_n'].min()} - {data['total_n'].max()}")
print(f"- Event rates: {data['event_rate'].min():.1%} - {data['event_rate'].max():.1%}")

In [ ]:
# Visualize study characteristics
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Publication timeline
axes[0].hist(data['year'], bins=10, alpha=0.7)
axes[0].set_xlabel('Publication Year')
axes[0].set_ylabel('Number of Studies')
axes[0].set_title('Study Publication Timeline')

# Sample size distribution
axes[1].hist(data['total_n'], bins=10, alpha=0.7)
axes[1].set_xlabel('Total Sample Size')
axes[1].set_ylabel('Number of Studies')
axes[1].set_title('Sample Size Distribution')

# Effect size vs sample size
axes[2].scatter(data['total_n'], data['effect_size'], alpha=0.7)
axes[2].set_xlabel('Total Sample Size')
axes[2].set_ylabel('Effect Size')
axes[2].set_title('Effect Size vs Sample Size')

plt.tight_layout()
plt.show()

## 3. Meta-Analysis

In [ ]:
# Fixed-effect meta-analysis
fixed_result = pm.meta_analysis(data, model='fixed')

print("Fixed-Effect Model:")
print(f"Overall effect: {fixed_result.overall_effect:.3f}")
print(f"95% CI: [{fixed_result.ci_lower:.3f}, {fixed_result.ci_upper:.3f}]")
print(f"p-value: {fixed_result.pvalue:.3f}")

In [ ]:
# Random-effects meta-analysis
random_result = pm.meta_analysis(data, model='random', tau2_method='reml')

print("Random-Effects Model (REML):")
print(f"Overall effect: {random_result.overall_effect:.3f}")
print(f"95% CI: [{random_result.ci_lower:.3f}, {random_result.ci_upper:.3f}]")
print(f"p-value: {random_result.pvalue:.3f}")
print(f"Tau²: {random_result.tau_squared:.3f}")
print(f"I²: {random_result.i_squared:.1f}%")

## 4. Heterogeneity Assessment

In [ ]:
# Test for heterogeneity
print(f"Cochran's Q: {random_result.q_statistic:.2f}")
print(f"Q p-value: {random_result.q_pvalue:.3f}")
print(f"I² statistic: {random_result.i_squared:.1f}%")

if random_result.q_pvalue < 0.05:
    print("\n✓ Significant heterogeneity detected")
    print("Random-effects model is appropriate")
else:
    print("\n→ No significant heterogeneity")
    print("Fixed-effect model may be sufficient")

In [ ]:
# Diagnostic plots for heterogeneity
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Baujat plot
pm.baujat_plot(random_result, ax=axes[0, 0], title='Baujat Plot')

# Radial plot
pm.radial_plot(random_result, ax=axes[0, 1], title='Radial Plot')

# L'Abbe plot
pm.labbe_plot(data, ax=axes[1, 0], title="L'Abbe Plot")

# GOSH plot
pm.gosh_plot(random_result, ax=axes[1, 1], title='GOSH Plot')

plt.tight_layout()
plt.show()

## 5. Publication Bias Assessment

In [ ]:
# Funnel plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Basic funnel plot
pm.funnel_plot(random_result, ax=axes[0], title='Funnel Plot')

# Funnel plot with contour lines
pm.funnel_plot(random_result, ax=axes[1], contours=True, title='Funnel Plot with Contours')

plt.tight_layout()
plt.show()

In [ ]:
# Statistical tests for publication bias
egger_result = pm.egger_test(data)
begg_result = pm.begg_test(data)

print("Publication Bias Tests:")
print("-" * 25)
print(f"Egger's test: p = {egger_result.pvalue:.3f}")
print(f"Begg's test: p = {begg_result.pvalue:.3f}")

if egger_result.pvalue < 0.05 or begg_result.pvalue < 0.05:
    print("\n⚠️  Evidence of publication bias detected")
else:
    print("\n✓ No significant evidence of publication bias")

In [ ]:
# Trim-and-fill analysis
tf_result = pm.trim_and_fill(data)

print(f"Trim-and-Fill Analysis:")
print("-" * 25)
print(f"Studies trimmed: {tf_result.n_trimmed}")
print(f"Studies filled: {tf_result.n_filled}")
print(f"Adjusted effect: {tf_result.adjusted_effect:.3f}")
print(f"Adjusted 95% CI: [{tf_result.ci_lower:.3f}, {tf_result.ci_upper:.3f}]")

# Plot trim-and-fill results
fig, ax = plt.subplots(figsize=(8, 6))
pm.trim_fill_plot(tf_result, ax=ax, title='Trim-and-Fill Analysis')
plt.show()

## 6. Sensitivity Analysis

In [ ]:
# Leave-one-out analysis
loo_result = pm.leave_one_out(data)

print("Leave-One-Out Analysis:")
print("-" * 25)
print(f"Effect size range: {loo_result.min_effect:.3f} to {loo_result.max_effect:.3f}")
print(f"Most influential study: {loo_result.most_influential_study}")
print(f"Influence measure: {loo_result.max_influence:.3f}")

# Plot influence analysis
fig, ax = plt.subplots(figsize=(10, 6))
pm.influence_plot(loo_result, ax=ax, title='Leave-One-Out Influence Analysis')
plt.show()

In [ ]:
# Cumulative meta-analysis
cum_result = pm.cumulative_analysis(data, order_by='year')

# Plot cumulative analysis
fig, ax = plt.subplots(figsize=(12, 6))
pm.cumulative_plot(cum_result, ax=ax, title='Cumulative Meta-Analysis by Publication Year')
plt.show()

## 7. Final Results and Forest Plot

In [ ]:
# Create publication-ready forest plot
fig, ax = plt.subplots(figsize=(12, 8))

forest_plot = pm.forest_plot(
    random_result,
    ax=ax,
    style='publication',
    title='Meta-Analysis of Intervention X vs Control',
    xlabel='Risk Ratio (95% CI)',
    show_weights=True,
    study_labels='left'
)

plt.tight_layout()
plt.show()

## 8. Summary and Interpretation

In [ ]:
# Generate summary table
summary = pm.create_summary_table([fixed_result, random_result], 
                                 labels=['Fixed-Effect', 'Random-Effects'])

print("Meta-Analysis Summary:")
print("=" * 50)
display(summary)

print("\nKey Findings:")
print("-" * 15)
print(f"• {len(data)} studies included ({data['total_n'].sum():,} participants)")
print(f"• Random-effects pooled estimate: {random_result.overall_effect:.3f} (95% CI: {random_result.ci_lower:.3f}, {random_result.ci_upper:.3f})")
print(f"• Moderate heterogeneity: I² = {random_result.i_squared:.1f}%")
print(f"• {('No significant' if egger_result.pvalue >= 0.05 else 'Evidence of')} publication bias")
print(f"• Results stable in sensitivity analyses")

## 9. Export Results

In [ ]:
# Export results to various formats
pm.export_results(random_result, 
                 formats=['csv', 'excel', 'json'],
                 filename='meta_analysis_results')

# Save forest plot
fig.savefig('forest_plot.png', dpi=300, bbox_inches='tight')
fig.savefig('forest_plot.pdf', bbox_inches='tight')

print("Results exported successfully!")
print("Files created:")
print("- meta_analysis_results.csv")
print("- meta_analysis_results.xlsx")
print("- meta_analysis_results.json")
print("- forest_plot.png")
print("- forest_plot.pdf")

## Conclusion

This notebook demonstrated a complete meta-analysis workflow using PyMeta:

1. **Data exploration** - Understanding study characteristics
2. **Effect size pooling** - Fixed and random-effects models
3. **Heterogeneity assessment** - Statistical tests and diagnostic plots
4. **Publication bias evaluation** - Funnel plots and statistical tests
5. **Sensitivity analysis** - Leave-one-out and cumulative analysis
6. **Results presentation** - Forest plots and summary tables
7. **Export functionality** - Multiple output formats

PyMeta provides a comprehensive toolkit for rigorous meta-analysis in Python, with publication-ready visualizations and robust statistical methods.